In [34]:
import pandas as pd
import re

# Preprocessing Football Manager Data

In [9]:
fm23Original = pd.read_excel("data/fm23data.xlsx")

In [18]:
fm23 = fm23Original.copy()
# print(fm23.head())
# list(fm23.columns.values)
# 
# #result: lots of whitespace

In [19]:
# strip column names from outer whitespace
fm23.columns = fm23.columns.str.strip()

# remove all but the following columns:
# Name, DoB, Age, Nat, Division, Club, Weight, Height, Position, Wage, Style, Tansfer_value, Preferred Foot, Best Role
fm23 = fm23.drop(columns = ['Inf', 'Based', 'Right Foot', 'Left Foot', 'AT Apps', 'AT Gls', 'Team', 'Best Role',
       'Caps', 'Yth Apps', 'Rc Injury', 'Best Duty', 'Best Pos', 'Acc', 'Aer', 'Agg', 'Agi', 'Ant', 'Bal', 'Bra', 'Cmd',
       'Com', 'Cmp', 'Cnt', 'Cor', 'Cro', 'Dec', 'Det', 'Dri', 'Ecc', 'Fin',
       'Fir', 'Fla', 'Fre', 'Han', 'Hea', 'Jum', 'Kic', 'Ldr', 'Lon', 'L Th',
       'Mar', 'Nat .1', 'OtB', '1v1', 'Pac', 'Pas', 'Pen', 'Pos', 'Pun', 'Ref',
       'TRO', 'Sta', 'Str', 'Tck', 'Tea', 'Tec', 'Thr', 'Vis', 'Wor'])
print(fm23.columns)
print(fm23.head())

Index(['UID', 'Name', 'DoB', 'Nat', 'Division', 'Club', 'Preferred Foot',
       'Position', 'Height', 'Weight', 'Age', 'Transfer Value', 'Wage',
       'Style', 'Best Role'],
      dtype='object')
        UID                                  Name  \
0    831079   Giorgio Chiellini                     
1   2105510   Hulk                                  
2  78000335   Luis Suárez                           
3    962988   Gonzalo Higuaín                       
4  19007730   Alex Teixeira                         

                           DoB    Nat  \
0   14/8/1984 (37 years old)     ITA    
1   25/7/1986 (35 years old)     BRA    
2   24/1/1987 (35 years old)     URU    
3   10/12/1987 (34 years old)    ARG    
4   6/1/1990 (32 years old)      BRA    

                                          Division  \
0   Major League Soccer                              
1   Brazilian National First Division                
2   Uruguayan First Division                         
3   Major League Soc

In [50]:
fm23 = fm23.astype({'Division': 'string', 'DoB': 'string', 'Nat': 'string', 'Name': 'string', 'Club': 'string', 'Preferred Foot': 'string', 'Position': 'string', 'Height': 'string', 'Weight': 'string', 'Transfer Value': 'string', 'Wage': 'string', 'Style': 'string', 'Best Role': 'string'})
fm23.dtypes

UID                        int64
Name              string[python]
DoB               string[python]
Nat               string[python]
Division          string[python]
Club              string[python]
Preferred Foot    string[python]
Position          string[python]
Height            string[python]
Weight            string[python]
Age                        int64
Transfer Value    string[python]
Wage              string[python]
Style             string[python]
Best Role         string[python]
dtype: object

In [21]:
# strip all entries from leading and trailing whitespace
fm23 = fm23.applymap(lambda x: x.strip() if isinstance(x, str) else x)
# check if strip actually worked and no results are returned in the case of division
fm23[fm23['Division']==" Major League Soccer                           "]

<ipython-input-21-a380bd8c6e7a>:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  fm23 = fm23.applymap(lambda x: x.strip() if isinstance(x, str) else x)


,UID,Name,DoB,Nat,Division,Club,Preferred Foot,Position,Height,Weight,Age,Transfer Value,Wage,Style,Best Role


In [104]:
leagues = ["Turkish Super League",
"Bundesliga",
"Bundesliga 2",
"Saudi Professional League",
"Jupiler Pro League",
"Italian Serie A",
"Italian Serie B",
"3. Liga",
"Ligue 1 Uber Eats",
"Ligue 2 BKT",
"Eredivisie",
"Portuguese Premier League",
"Major League Soccer",
"Chinese Super League",
"Sky Bet Championship",
"English Premier Division",
"Spanish First Division",
"Spanish Second Division"]

fm23_sub = fm23[fm23['Division'].isin(leagues)] 
fm23_sub


,UID,Name,DoB,Nat,Division,Club,Preferred Foot,Position,Height,Weight,Age,Transfer Value,Wage,Style,Best Role
0,831079,Giorgio Chiellini,14/8/1984 (37 years old),ITA,Major League Soccer,LAFC,Left Only,D (C),187 cm,85 kg,37,Not for Sale,"€90,500 p/m",Leader,Central Defender
3,962988,Gonzalo Higuaín,10/12/1987 (34 years old),ARG,Major League Soccer,Inter Miami,Right,ST (C),184 cm,89 kg,34,€90K - €900K,"€487,000 p/m",Intelligent,Deep Lying Forward
7,5132312,Gareth Bale,16/7/1989 (32 years old),WAL,Major League Soccer,LAFC,Left,"M (L), AM (RLC), ST (C)",185 cm,82 kg,32,€550K - €5.6M,"€135,000 p/m",Creative,Inverted Winger
8,51000001,Carlos Vela,1/3/1989 (33 years old),MEX,Major League Soccer,LAFC,Left Only,"AM (RLC), ST (C)",178 cm,74 kg,33,Not for Sale,"€340,000 p/m",Creative,Inverted Winger
9,43043933,Lorenzo Insigne,4/6/1991 (31 years old),ITA,Major League Soccer,Toronto FC,Right,"AM (L), ST (C)",163 cm,59 kg,31,Not for Sale,"€1,176,000 p/m",Creative,Inverted Winger
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189262,2000225262,José Bolivar,26/8/2005 (16 years old),VEN,Portuguese Premier League,VTSC,Right,GK,190 cm,80 kg,16,0,€275 p/m,Assertive,Sweeper Keeper
189265,2000225265,Franck-Loïc Zoumessi,20/1/2004 (18 years old),CAN,Portuguese Premier League,BRG,Right,M/AM (R),197 cm,87 kg,18,0,€275 p/m,Leader,Winger
189266,2000225266,Guilherme Costa,3/9/2005 (16 years old),BRA,Portuguese Premier League,BRG,Right,AM (C),171 cm,58 kg,16,€0 - €60K,€275 p/m,Creative,Attacking Midfielder
189267,2000225267,Steven Roßbach,19/8/2004 (17 years old),GER,Portuguese Premier League,VIZ,Right Only,D (C),193 cm,79 kg,17,€500 - €3.5K,€275 p/m,Technical,Central Defender


In [105]:
#double check
unique_leagues = fm23_sub['Division'].unique()
print(unique_leagues)

<StringArray>
[      'Major League Soccer',                'Bundesliga',
                'Eredivisie',        'Jupiler Pro League',
              'Bundesliga 2',                   '3. Liga',
           'Italian Serie A',           'Italian Serie B',
      'Turkish Super League',      'Chinese Super League',
 'Saudi Professional League',  'English Premier Division',
      'Sky Bet Championship',         'Ligue 1 Uber Eats',
    'Spanish First Division', 'Portuguese Premier League',
   'Spanish Second Division',               'Ligue 2 BKT']
Length: 18, dtype: string


## Further Preprocessing
1. Weight (remove kg)
2. Height (remove cm)
3. DoB (remove the age in ())
4. Wage (remove € and p/m)
5. Preferred Foot (remove Only)
6. Transfer Value (remove €, replace "Not for Sale" with NaN and calculate mean of the given range)


In [106]:
fm23_sub['Weight'] = fm23_sub['Weight'].str.strip(" kg")

<ipython-input-106-10e6a9426284>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fm23_sub['Weight'] = fm23_sub['Weight'].str.strip(" kg")


### Weight, Height, Wage

In [107]:
def remove_cm(string):
    return string.replace(" cm", "")

def remove_kg(string):
    return string.replace(" kg", "")

def wage(string):
    return string.replace("€", "").replace(" p/m", "")

In [108]:
fm23_sub['Height'] = fm23_sub['Height'].apply(remove_cm)
fm23_sub['Weight'] = fm23_sub['Weight'].apply(remove_kg)
fm23_sub['Wage'] = fm23_sub['Wage'].apply(wage)

<ipython-input-108-75790dae3a78>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fm23_sub['Height'] = fm23_sub['Height'].apply(remove_cm)
<ipython-input-108-75790dae3a78>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fm23_sub['Weight'] = fm23_sub['Weight'].apply(remove_kg)
<ipython-input-108-75790dae3a78>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pan

### DOB

In [109]:
fm23_sub["DoB"] = fm23_sub["DoB"].apply(lambda x: re.sub(r'\([^)]*\)', '', x))

<ipython-input-109-6c274df60c7f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fm23_sub["DoB"] = fm23_sub["DoB"].apply(lambda x: re.sub(r'\([^)]*\)', '', x))


### Preferred Foot

In [110]:
fm23_sub['Preferred Foot'] = fm23_sub['Preferred Foot'].apply(lambda x: x.replace("Only", ""))

<ipython-input-110-17a3100b2228>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fm23_sub['Preferred Foot'] = fm23_sub['Preferred Foot'].apply(lambda x: x.replace("Only", ""))


### Transfer Value

In [111]:
fm23_sub['Transfer Value'] = fm23_sub['Transfer Value'].apply(lambda x: x.replace("€", ""))

<ipython-input-111-880e78d33111>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fm23_sub['Transfer Value'] = fm23_sub['Transfer Value'].apply(lambda x: x.replace("€", ""))


In [125]:
def get_market_value(input_string):
    
    market_value = []

    for i in input_string:
        tokens = re.split(r' \- ', i)
        #this list is needed to store the potential start and end of the range of the market value to later calculate average
        inner_list = []

        if "Not for Sale" not in tokens:
            
            for token in tokens:
                token = token.replace("K", "000")

                #if there is "." (only exists for Millions) then replace M with only five 0s and remove dot
                if "." in token:
                    token = token.replace("M", "00000").replace(".", "")
                #else replace with six 0s
                else:
                    token = token.replace("M", "000000")
                
                inner_list.append(token)
            
            market_value.append(inner_list)

        else:
            market_value.append(None)

    return market_value

In [140]:
all_values = get_market_value(fm23_sub['Transfer Value'])

final_values = []

for i in all_values:
    if i != None:
        if len(i) > 1:
            avg = (int(i[0]) + int(i[1]))/2
            final_values.append(avg)
        else:
            final_values.append(i[0])
    else:
        final_values.append(None)

final_values

fm23_sub['Transfer Value'] = final_values

<ipython-input-140-18b76c1b0787>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fm23_sub['Transfer Value'] = final_values


In [141]:
fm23_sub.head()

,UID,Name,DoB,Nat,Division,Club,Preferred Foot,Position,Height,Weight,Age,Transfer Value,Wage,Style,Best Role
0,831079,Giorgio Chiellini,14/8/1984,ITA,Major League Soccer,LAFC,Left,D (C),187,85,37,None,"90,500",Leader,Central Defender
3,962988,Gonzalo Higuaín,10/12/1987,ARG,Major League Soccer,Inter Miami,Right,ST (C),184,89,34,495000.0,"487,000",Intelligent,Deep Lying Forward
7,5132312,Gareth Bale,16/7/1989,WAL,Major League Soccer,LAFC,Left,"M (L), AM (RLC), ST (C)",185,82,32,3075000.0,"135,000",Creative,Inverted Winger
8,51000001,Carlos Vela,1/3/1989,MEX,Major League Soccer,LAFC,Left,"AM (RLC), ST (C)",178,74,33,None,"340,000",Creative,Inverted Winger
9,43043933,Lorenzo Insigne,4/6/1991,ITA,Major League Soccer,Toronto FC,Right,"AM (L), ST (C)",163,59,31,None,"1,176,000",Creative,Inverted Winger


In [142]:
fm23_sub.to_csv("data/fm23_preprocessed.csv", index=False)